In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from pydrake.all import StartMeshcat, RotationMatrix, RigidTransform
from matplotlib import pyplot as plt

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
goal_rotation = RotationMatrix(
    [
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1],
    ]
)
goal_position = np.array([-1, -1, 1])
goal_pose = RigidTransform(goal_rotation, goal_position)

In [ ]:
# from mobile_base_ik import solve_ik, set_position
from mobile_iiwa import set_position
from IPython.display import display, Image, SVG
import pydot
from matplotlib import pyplot as plt

In [ ]:
# gripper_poses = [RigidTransform([0.5, 0, 0.5]), RigidTransform([0.5, 0, 0]), RigidTransform([0.5, 0, 0]), RigidTransform([0.5, 0, 0]), RigidTransform([0.5, 0, 0.5]), RigidTransform([0, -0.3, 0.5]), RigidTransform([-0.5, -0.3, 0.5]), RigidTransform([-.5, 0, 0.5]), RigidTransform([-.5, 0, 0.5])] 
base_pos = (1.5, 1)
diagram, station = set_position(meshcat, base_pos=base_pos)
SVG(
    pydot.graph_from_dot_data(diagram.GetGraphvizString(max_depth=1))[
        0
    ].create_svg())

In [ ]:
system = diagram
context = system.CreateDefaultContext()
system.ForcedPublish(context)
for i in range(3):
    color_image = system.GetOutputPort(f"camera{i}_rgb_image").Eval(context)
    depth_image = system.GetOutputPort(f"camera{i}_depth_image").Eval(context)

    # Plot the two images.
    plt.subplot(121)
    plt.imshow(color_image.data)
    plt.title(f"camera{i} Color image")
    plt.subplot(122)
    plt.imshow(np.squeeze(depth_image.data))
    plt.title(f"camera{i} Depth image")
    # mpld3.display()
    plt.show()

In [ ]:
system = diagram
context = system.CreateDefaultContext()
system.ForcedPublish(context)
for i in range(3):
    point_cloud = system.GetOutputPort(f"camera{i}_point_cloud").Eval(context)

    # Plot the two images.
    plt.subplot(121)
    plt.imshow(point_cloud.data)
    plt.title(f"camera{i} point cloud")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99019c02-9e81-448d-8532-b07e66cf3903' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>